In [1]:
from utensor_cgen.api.export import tflm_keras_export

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import os
import cv2
import random

#dirname = os.path.dirname(__file__)
DATADIR = os.path.join(os.getcwd(), 'Dataset')

#DATADIR = "C:/Users/jerickson/Documents/CS688/Dataset/Dataset"
CATEGORIES = ["with_mask","without_mask"]

all_data = []

IMG_SIZE = 28

def create_all_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                new_array = new_array / 255.0
                all_data.append([new_array, class_num])
            except Exception as e:
                pass

create_all_data()

In [3]:
all_data_len = int(len(all_data))

print(all_data_len)

random.shuffle(all_data)

test_size = int(all_data_len * 0.2)

slice_1 = all_data_len - test_size
slice_2 = -1*test_size
train_data = all_data[:slice_1]
test_data = all_data[slice_2:]

train_features = []
train_label = []

test_features = []
test_label = []

for features,label in train_data:
    train_features.append(features)
    train_label.append(label)
    


for features,label in test_data:
    test_features.append(features)
    test_label.append(label)

    
train_features = np.array(train_features).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_features = np.array(test_features).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


train_ds = tf.data.Dataset.from_tensor_slices(
    (train_features, train_label)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_label)).batch(32)


11042


In [4]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

tf.keras.backend.clear_session()
print("TensorFlow Version: " + tf.__version__)

TensorFlow Version: 2.1.0


In [5]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(8, 3, activation='relu')
    self.pool = MaxPooling2D()
    self.flatten = Flatten()
    self.d1 = Dense(16, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x0 = self.pool(x)
    x1 = self.conv1(x0)
    x2 = self.pool(x1)
    x3 = self.flatten(x2)
    x4 = self.d1(x3)
    return self.d2(x4)

model = MyModel()

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 15

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.833448052406311, Accuracy: 65.11206817626953, Test Loss: 0.5617481470108032, Test Accuracy: 76.49456024169922
Epoch 2, Loss: 0.5225998759269714, Accuracy: 76.9753189086914, Test Loss: 0.48930707573890686, Test Accuracy: 76.81159210205078
Epoch 3, Loss: 0.47005510330200195, Accuracy: 79.18270111083984, Test Loss: 0.4426388740539551, Test Accuracy: 80.52536010742188
Epoch 4, Loss: 0.44354763627052307, Accuracy: 80.45053100585938, Test Loss: 0.4155847132205963, Test Accuracy: 82.4275360107

In [11]:
# representative data function
num_calibration_steps = 128
calibration_dtype = tf.float32

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        rand_idx = np.random.randint(0, test_features.shape[0]-1)
        sample = test_features[rand_idx]
        sample = sample[tf.newaxis, ...]
        sample = tf.cast(sample, dtype=calibration_dtype)
        yield [sample]

In [12]:
tflm_keras_export(
    model,
    representive_dataset=representative_dataset_gen,
    model_name='my_model',
    target='utensor',
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: C:\Users\qrrt_\AppData\Local\Temp\utensor_zl7ta5tp\saved_model\assets
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING quantize.py <module> @ 12] trying to import deprecated quantization transformer
[INFO transformer.py transform @ 23] Transforming graph: my_model
[INFO transformer.py transform @ 24] Transform pipeline: dropout(name_pattern=r'(dropout[_\w\d]*)/.*') -> linear_reorder -> inline -> biasAdd -> remove_id_op -> fake_gather_v2 -> refcnt
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_post_init__ @ 300] cannot convert new_shape to generic value: [1, 288](<class 'list'>)
[WARNING base.py __attrs_